# **GradientBoostingRegressor**

*Fonctionne avec Python 3.10.9 (Anaconda 23.3.1)*

## **Importation des bibliothèques**

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pour les traîtements sur les variables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modèle de ML utilisé
from sklearn.ensemble import GradientBoostingRegressor

# Pour gridsearch
from sklearn import model_selection

# Pour mesures des metrics des résultats
from sklearn import metrics

# Pour mise en forme des résultats
import colorama

## **Importation du dataset**

In [37]:
dataset_a_utiliser = "data_clean.xlsx"

data = pd.read_excel(dataset_a_utiliser)

## **Fonctions**

In [38]:
def split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification):

    ##############################
    # Split du dataset en X et y #
    ##############################

    global X
    global y

    X = data.loc[:,features_list] # On ajoute nos features dans le X
    y = data.loc[:,y_column] # On ajoute ce qu'on veut prédire dans le y

    ############################
    # Scaling et Encoding de X #
    ############################

    global feature_encoder

    # Ici nous avons de meilleurs résultats avec RobustScaler()
    numeric_transformer = StandardScaler()

    categorical_transformer = OneHotEncoder()

    feature_encoder = ColumnTransformer(
                transformers=[
                    ('cat', categorical_transformer, categorical_features),    
                    ('num', numeric_transformer, numeric_features)
                    ]
                )
    X = feature_encoder.fit_transform(X)


    ####################################
    # Split de X et y en train et test #
    ####################################

    global X_train
    global X_test
    global y_train
    global y_test

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.2, 
                                                        random_state=0,
                                                        stratify=eval(stratification))

## **Features**

In [39]:
######################
# Choix des features #
######################

dataset_name = data

# Si rien mettre []
categorical_features = [
                        #'BuildingType',
                        'PrimaryPropertyType',
                        'Neighborhood',
                        'ZipCode',
                        #'YearBuilt' # Meilleurs résultats sans
                       ]

# Si rien mettre []
numeric_features = [
                    'NumberofBuildings',
                    #'NumberofFloors', # Meilleurs résultats sans
                    'PropertyGFAParking',
                    'PropertyGFABuilding(s)',
                    'Latitude',
                    'Longitude',
                    'age_bat'
                   ]

# Toutes les features
features_list = categorical_features + numeric_features

## ***GradientBoostingRegressor***

In [40]:
# Targets à estimer
targets = ['SiteEnergyUse_kBtu', 'TotalGHGEmissions']

# "y" pour stratifier y, sinon "None"
stratification = "None"

# Afficher les résultats détaillés (True/False)
details = False

for i in targets:

    y_column=i

    print(f"=========== [{i}] ===========")


    # Split & Scale du dataset
    split_and_scale(dataset_name, y_column, features_list, numeric_features, categorical_features, stratification)


    # Entraînement simple
    modele = GradientBoostingRegressor() # Paramètres par défaut
    modele.fit(X_train, y_train) # Étape d'entraînement
    print("--- Entraînement simple : ---")
    print(f"Scores du modèle pour prédire {i} :")
    y_pred = modele.predict(X_test)
    print(f"R² : {modele.score(X_train, y_train):.3f} (train) et {colorama.Style.BRIGHT}{colorama.Back.CYAN}{colorama.Fore.BLACK} {modele.score(X_test, y_test):.3f} {colorama.Style.RESET_ALL} (test)")
    print(f"MSE : {metrics.mean_squared_error(y_test, y_pred):.4}")
    print(f"MAE : {metrics.mean_absolute_error(y_test, y_pred):.4}")


    ###############################################
    # Paramétrage de GridSearchCV et entraînement #
    ###############################################

    print("--- GridSearch : ---")

    alphas = np.linspace(0.8, 0.87, 5)

    # Fixer les valeurs des hyperparamètres à tester
    param_grid = [
                  {'loss':['huber', 'quantile'], 'alpha':[0.8, 0.835], 'n_estimators':[200], 'learning_rate':[0.17]}
                  # 'huber', 'quantile' : ces deux donnent les meilleurs résultats
                  # n_estimators=200 (par défaut 100) : améliore un peu les scores, sans trop augmenter le temps
                  # learning_rate=0.17 : après essais, donne les meilleurs résultats
                  # alpha : 0.835 et 0.8 sont ceux qui donnent les meilleurs résultats
                 ]
    # learning_rate=0.001, n_estimators=100

    # Déterminer le score qu'on veut optimiser
    score = 'r2'

    # Je le sors ici car je vais réutiliser cette valeur plus tard
    cv = 5

    grid = model_selection.GridSearchCV(
        GradientBoostingRegressor(), # On indique le modèle à tester
        param_grid,     # hyperparamètres à tester
        cv=cv,           # nombre de folds de validation croisée
        scoring=score   # score à optimiser
    )

    # Optimiser ce modèle sur le jeu d'entraînement
    grid.fit(X_train, y_train)


    ###########################
    # Affichage des résultats #
    ###########################

    # Afficher le(s) hyperparamètre(s) optimaux
    print(f"Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement ({i}): {grid.best_params_}")

    # Afficher les performances correspondantes
    if details == True:
        print('Résultats pour chaque fold :')
        for j in range(cv):
            print(f"Fold n°{j+1} :", eval(f"grid.cv_results_['split{j}_test_score']"), f"(Pour les paramètres : {grid.cv_results_['params']})")

        print("\nRésultats de la validation croisée :")
        for mean, std, params in zip(
                grid.cv_results_['mean_test_score'], # score moyen
                grid.cv_results_['std_test_score'],  # écart-type du score
                grid.cv_results_['params']           # valeur de l'hyperparamètre
            ):

            print(f"{score} (moyen) : {mean:.04f} (+/-{std*2:.04f}) pour {params}")


    ###################################################################
    # Prédiction sur le jeu de test avec les hyperparamètres optimaux #
    ###################################################################

    # GridSearchCV a automatiquement ré-entraîné le meilleur modèle sur l’intégralité du jeu d’entraînement.
    y_pred = grid.predict(X_test)
    print(f"Score sur le jeu de test ({i}, avec paramètres optimaux) :")
    print(f"R² : {colorama.Style.BRIGHT}{colorama.Back.GREEN}{colorama.Fore.BLACK} {metrics.r2_score(y_test, y_pred):.3f} {colorama.Style.RESET_ALL}")
    print(f"MSE : {metrics.mean_squared_error(y_test, y_pred):.4}")
    print(f"MAE : {metrics.mean_absolute_error(y_test, y_pred):.4}")

    print('_'*80, "\n")

    

=========== [SiteEnergyUse_kBtu] ===========
--- Entraînement simple : ---
Scores du modèle pour prédire SiteEnergyUse_kBtu :
R² : 0.908 (train) et  0.710  (test)
MSE : 5.259e+13
MAE : 3.319e+06
--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (SiteEnergyUse_kBtu): {'alpha': 0.8, 'learning_rate': 0.17, 'loss': 'huber', 'n_estimators': 200}
Score sur le jeu de test (SiteEnergyUse_kBtu, avec paramètres optimaux) :
R² :  0.706 
MSE : 5.346e+13
MAE : 3.003e+06
________________________________________________________________________________ 

=========== [TotalGHGEmissions] ===========
--- Entraînement simple : ---
Scores du modèle pour prédire TotalGHGEmissions :
R² : 0.913 (train) et  0.572  (test)
MSE : 4.08e+04
MAE : 95.41
--- GridSearch : ---
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement (TotalGHGEmissions): {'alpha': 0.8, 'learning_rate': 0.17, 'loss': 'quantile', 'n_estimators': 200}
Score sur le jeu de test (TotalGHGEmissions, avec paramètres op